In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from scipy.signal import find_peaks, peak_widths
import skimage

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
INPUT_DATA_PATH = r"../../data/dataset_train.csv"

# Metro in Porto, Portugal

## Data

### Description

[data](https://zenodo.org/record/6854240#.YvuPmHZBzBV)

[paper](https://arxiv.org/pdf/2207.05466.pdf#view=fitH&toolbar=1)

Notes:
- Meta data is stored in an external excel file.
- Digital variables assume only two values: `0` when inactive or `1` when activated.

In [4]:
# img = skimage.io.imread("images/2022-08-26 15_07_17.png")
# fig = px.imshow(img, title="Pressure air unit")
# fig.update_traces(hovertemplate=None, hoverinfo="skip")
# fig.update_layout(margin=dict(l=20, r=20, t=40, b=20))
# fig.update_xaxes(visible=False)
# fig.update_yaxes(visible=False)
# fig.show()

### Columns

In [5]:
features_info = pd.read_excel("meta_data.xlsx", sheet_name = "features")
columns_to_hide = ["keep", "new_name", "description_long"]
features_info[["original_name", "variable_type", "description_short"]]

FileNotFoundError: [Errno 2] No such file or directory: 'meta_data.xlsx'

### Failures

In [ ]:
reported_failures = pd.read_excel("meta_data.xlsx", sheet_name = "failures")
reported_failures

## Preprocess

### Read

In [ ]:
metro = pd.read_csv(INPUT_DATA_PATH)

### Get to know

In [ ]:
metro.head(2)

In [ ]:
metro.shape

In [ ]:
metro.columns

In [ ]:
metro.dtypes

### Check for missing values

In [ ]:
metro.isna().any()

### Select columns

In [ ]:
columns_to_keep = features_info[features_info.keep == "yes"].original_name
metro = metro[columns_to_keep]

### Rename columns

In [ ]:
new_names = features_info[features_info.keep == "yes"].new_name
new_names.name = ""
metro.columns = new_names

### Set index

In [ ]:
metro.timestamp = pd.to_datetime(metro.timestamp)
metro = metro.set_index("timestamp")

In [ ]:
metro.index[0], metro.index[-1]

## Explore time series

### Define column groups

In [ ]:
used_columns = features_info[features_info.keep == "yes"]
columns_by_type = used_columns.groupby("variable_type")
columns_by_type.groups

In [ ]:
analogue_columns = columns_by_type.get_group("analogue").new_name
digital_columns = columns_by_type.get_group("digital").new_name
gps_columns = columns_by_type.get_group("gps data").new_name

all_columns = np.concatenate([analogue_columns, digital_columns])

### Examine a single day

In [ ]:
def plot_multiple_columns(df, cols, title=""):
    num_plots = len(cols)
    fig, axs = plt.subplots(num_plots, figsize = (8, 2 * num_plots), sharex=True)
    for i, col in enumerate(cols):
        axs[i].plot(df[col])
        axs[i].set_ylabel(col)
        axs[i].set_title(col)
    axs[-1].set_xlabel("Time")
    fig.suptitle(title)
    fig.tight_layout()
    plt.show()

In [ ]:
single_day = "('2022-01-10 08:00' < timestamp) & (timestamp < '2022-01-10 23:00')"
plot_multiple_columns(metro.query(single_day), analogue_columns)

### Examine a morning

In [ ]:
morning = "('2022-01-10 08:00' < timestamp) & (timestamp < '2022-01-10 10:00')"
plot_multiple_columns(metro.query(morning), all_columns)

### Examine the first failure

In [ ]:
# Info
reported_failures[reported_failures.id == 1]

In [ ]:
failure_interval = "('2022-02-28 21:00' <= timestamp) & (timestamp <= '2022-03-01 03:00')"
first_failure = metro.query(failure_interval)

plot_multiple_columns(first_failure, analogue_columns)

### Examine the second failure

In [ ]:
# Info
reported_failures[reported_failures.id == 2]

In [ ]:
failure_interval = "('2022-03-23 14:00' <= timestamp) & (timestamp <= '2022-03-23 16:00')"
second_failure = metro.query(failure_interval)

plot_multiple_columns(second_failure, analogue_columns)

### Examine the third failure

In [ ]:
# Info
reported_failures[reported_failures.id == 3]

In [ ]:
failure_interval = "('2022-05-30 11:00' <= timestamp) & (timestamp <= '2022-06-02 07:00')"
third_failure = metro.query(failure_interval)

plot_multiple_columns(third_failure, analogue_columns)

In [ ]:
failure_interval = "('2022-06-02 01:00' <= timestamp) & (timestamp <= '2022-06-02 07:00')"
third_failure_2 = metro.query(failure_interval)

plot_multiple_columns(third_failure_2, analogue_columns)

## Explore geographic data

### Plot train speed

In [ ]:
metro[gps_columns].head(2)

In [ ]:
start = "2022-01-10 08:00"
end = "2022-01-10 12:00"
query = f"('{start}' <= timestamp) & (timestamp <= '{end}') & (quality == 1)"
gps_data = metro[gps_columns].query(query)

In [ ]:
# fig = px.scatter_mapbox(
#     gps_data, 
#     lat="lat", 
#     lon="lon", 
#     color = "speed", 
#     # zoom=10, 
#     title=f"Metro route and speed from {start} to {end}",
# )

# fig.update_layout(mapbox_style="carto-positron")
# fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
# fig.show()

### Read station locations

In [ ]:
stations = pd.read_excel("meta_data.xlsx", sheet_name = "stations")
stations

### Plot train speed and stations

Observations:
- Low train speed indicates a station. The stations match with the indication.

In [ ]:
# fig = go.Figure()

# fig.add_trace(
#     go.Scattermapbox(
#         lat=gps_data.lat,
#         lon=gps_data.lon,
#         marker = go.scattermapbox.Marker(color = gps_data.speed),
#         name = "Train speed",
#     )
# )

# fig.add_trace(
#     go.Scattermapbox(
#         lat=stations.lat,
#         lon=stations.lon,
#         mode='markers',
#         marker=go.scattermapbox.Marker(
#             size=16,
#             color='green',
#             opacity=0.6,
#             ),
#         text=stations.station,
#         name = "Stations",
#     )
# )

# fig.update_layout(
#     mapbox_style="carto-positron",
#     margin={"r":0,"t":30,"l":0,"b":0},
#     title="Train speed and stations",
# )
# fig.show()

### Plot failure locations

Select narrower failure intervals based on the abnormalities in sensor readings observed in above analyses.

Also filter out the invalid gps readings (`[metro.quality == 1]`).

The third failure is omitted since:
- there is no valid GPS data in the failure period
- the time of occurrence can't be determined as the reported duration is much longer than the other two (2 days vs 1-2 hours)

In [ ]:
interval = "('2022-02-28 22:00' <= timestamp) & (timestamp <= '2022-03-01 01:00')"
first_breakdown = metro[metro.quality == 1].query(interval)

interval = "('2022-03-23 14:54' <= timestamp) & (timestamp <= '2022-03-23 15:24')"
second_breakdown = metro[metro.quality == 1].query(interval)

for breakdown in [first_breakdown, second_breakdown]:
    print(breakdown.shape)

In [ ]:
# fig = go.Figure()

# # Failures
# for breakdown, label in zip([first_breakdown, second_breakdown], ['First failure', 'Second failure']):
#     fig.add_trace(
#         go.Scattermapbox(
#             lat=breakdown.lat,
#             lon=breakdown.lon,
#             opacity=0.5,
#             marker={"size": 5,},
#             name=label,
#         )
#     )

# # Stations
# fig.add_trace(
#     go.Scattermapbox(
#         lat=stations.lat,
#         lon=stations.lon,
#         marker={
#             "size": 15,
#         },
#         text = stations.station,
#         name = "Stations"
#     )
# )

# fig.update_layout(
#     mapbox_style="carto-positron",
#     margin={"r":0,"t":30,"l":0,"b":0},
#     title="Failure and station locations"
# )
# fig.show()

### Check GPS data quality

In [ ]:
interval = "('2022-02-10 01:00' <= timestamp) & (timestamp <= '2022-06-02 06:00')"
quality_third_failure = metro.query(interval).quality

In [ ]:
plt.plot(quality_third_failure)
plt.show()

# TODO: threshold by frequency to remove noise

### Identify the peaks for one sensor signal

Steps:
1. Filter data for a short period and with proper GPS quality.
2. Select one analogue sensor signal. Find its peaks.
3. Plot to validate result.

`compressor_pressure` is chosen as signal because its shape seems suitable.

In [ ]:
start = "2022-01-10 08:00"
end = "2022-01-10 12:00"
one_hour = f"('{start}' <= timestamp) & (timestamp <= '{end}') & (quality == 1)"
metro_one_hour = metro.query(one_hour)

In [ ]:
signal = metro_one_hour.compressor_pressure
peak_indeces, _ = find_peaks(signal, height=10, distance=300)
peaks = signal.iloc[peak_indeces].copy()

In [ ]:
print(f"Peaks count: {len(peaks)}")

plt.figure(figsize = (12, 4))
plt.plot(signal)
plt.plot(peaks, "rx")
plt.show()

In [ ]:
# fig1 = px.line(signal)
# fig2 = px.scatter(peaks, color_discrete_sequence=['#ff7f0e'])
# fig3 = go.Figure(data=fig1.data + fig2.data)
# fig3.show()

### Compare peek locations to stations


In [ ]:
peaks_gps_data = metro_one_hour[metro_one_hour.index.isin(peaks.index)][["lon", "lat"]]

stations_and_peaks = pd.concat([stations, peaks_gps_data], keys=["station", "peak"])
stations_and_peaks = stations_and_peaks.reset_index()
stations_and_peaks = stations_and_peaks.drop(columns=["level_1"])
stations_and_peaks = stations_and_peaks.rename(columns={"level_0": "location_type"})

stations_and_peaks

In [ ]:
# fig = px.scatter_mapbox(
#     data_frame = stations_and_peaks,
#     lat="lat",
#     lon="lon",
#     color="location_type",
# )

# fig.update_layout(
#     mapbox_style="carto-positron",
#     margin={"r":0,"t":30,"l":0,"b":0},
#     title="Peeks and stations"
# )
# fig.show()

In [ ]:
# fig = go.Figure()

# # Sensor peaks
# fig.add_trace(
#     go.Scattermapbox(
#         lat=peaks_gps_data.lat,
#         lon=peaks_gps_data.lon,
#         name = "Compressor pressure peaks",
#     )
# )

# # Stations
# fig.add_trace(
#     go.Scattermapbox(
#         lat=stations.lat,
#         lon=stations.lon,
#         mode="markers",
#         marker={
#             "size": 15,
#         },
#         text = stations.station,
#         name = "Stations"
#     )  
# )

# fig.update_layout(
#     mapbox_style="carto-positron",
#     margin={"r":0,"t":30,"l":0,"b":0},
#     title="Failure and station locations"
# )
# fig.show()

### Plot widths
`peak_widths` returns a `tuple(width, y, x_start, x_end)`

In [ ]:
widths = peak_widths(x, peaks, rel_height=0.95)
widths 

In [ ]:
plt.plot(x)
plt.plot(peaks, x[peaks], "x")
plt.hlines(y = widths[1], xmin = widths[2], xmax = widths[3], color="red")
plt.xlim((1000, 1500))
plt.show()

In [ ]:
fig1 = px.line(x)
fig2 = px.scatter(x = peaks, y = x[peaks], color_discrete_sequence=['#ff7f0e'])
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()

In [ ]:
plt.plot(metro.Reservoirs)
plt.show()

In [ ]:
metro

### TODO compare diff days for same location / train stop